In [1]:
# Cluster in space

from eqcorrscan.utils.clustering import catalog_cluster
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

client = Client("IRIS")
starttime = UTCDateTime("2002-01-01")
endtime = UTCDateTime("2002-02-01")
cat = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=6, catalog="ISC")
groups = catalog_cluster(
    catalog=cat, metric="distance", thresh=1000, show=False)

In [2]:
# Download a local catalog of earthquakes and cluster much finer

client = Client("NCEDC")
cat = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=2)
groups = catalog_cluster(
    catalog=cat, metric="distance", thresh=2, show=False)

In [3]:
# Cluster in time and space

from eqcorrscan.utils.clustering import space_time_cluster
client = Client("IRIS")
cat = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=6, catalog="ISC")
groups = space_time_cluster(catalog=cat, t_thresh=86400, d_thresh=1000)

In [4]:
# Cluster according to cross-correlation values
from obspy import read
import glob
import os
from eqcorrscan.utils.clustering import cluster
from eqcorrscan import tests

# You will need to edit this line to the location of your eqcorrscan repo.
TEST_PATH = os.path.dirname(tests.__file__)
testing_path = TEST_PATH + '/test_data/similar_events_processed'
stream_files = glob.glob(os.path.join(testing_path, '*'))
stream_list = [(read(stream_file), i)
               for i, stream_file in enumerate(stream_files)]
for stream in stream_list:
    for tr in stream[0]:
        if tr.stats.station not in ['WHAT2', 'WV04', 'GCSZ']:
            stream[0].remove(tr) 
            continue
groups = cluster(template_list=stream_list, show=False,
                 corr_thresh=0.3, cores=2)

ValueError: array must be at least 2-d

In [ ]:
# Stack waveforms (linear)

from eqcorrscan.utils.stacking import linstack

# groups[0] should contain 3 streams, which we can now stack
# Groups are returned as lists of tuples, of the stream and event index
group_streams = [st_tuple[0] for st_tuple in groups[0]]
stack = linstack(streams=group_streams)

In [ ]:
# Stack waveforms (phase-weighted)

from eqcorrscan.utils.stacking import PWS_stack

# groups[0] should contain 3 streams, which we can now stack
# Groups are returned as lists of tuples, of the stream and event index
stack = PWS_stack(streams=group_streams)